In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as sp

In [5]:
import os
import joblib
from config import DATA_PATH
data_path = DATA_PATH

In [6]:
app_events = pd.read_csv(os.path.join(data_path, 'app_events.csv'), usecols=('is_installed', 'event_id', 'app_id'),
                         dtype={'is_installed': np.int8, 'event_id': np.int64, 'app_id': np.int64}, chunksize=1000)

In [8]:
all_apps = joblib.load('all_apps.joblib')

c:\python27\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: The file 'all_apps.joblib' has been generated with a joblib version less than 0.10. Please regenerate this pickle file.
  if __name__ == '__main__':


In [9]:
res = sp.csr_matrix((1488096, 19237))
events = []

In [11]:
current_event_id = None
item = None

counter = 0
v = None

for chunk in app_events:
    for row in chunk.values:
        event_id, app_id = row[0], row[1]
        if current_event_id is None:
            current_event_id = event_id
            item = dict(zip(all_apps, np.zeros(len(all_apps))))
        if current_event_id == event_id:
            item[app_id] = 1
        else:
            v = pd.DataFrame(item.items(), columns=('app', 'is')).set_index('app').T.values[0]
            res = sp.vstack([res, v])

            current_event_id = event_id
            item = dict(zip(all_apps, np.zeros(len(all_apps))))
            events.append(current_event_id)
        counter += 1

        if counter % 50000 == 0:
            print counter

if item is not None:
    v = pd.DataFrame(item.items(), columns=('app', 'is')).set_index('app').T.values[0]
    res = sp.hstack([res, v])

50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000
600000
650000
700000
750000
800000
850000
900000
950000
1000000
1050000
1100000
1150000
1200000
1250000
1300000
1350000
1400000
1450000
1500000
1550000
1600000
1650000
1700000
1750000
1800000
1850000
1900000
1950000
2000000
2050000
2100000
2150000
2200000
2250000
2300000
2350000
2400000
2450000
2500000
2550000
2600000
2650000
2700000
2750000
2800000
2850000
2900000
2950000
3000000
3050000
3100000
3150000
3200000
3250000
3300000
3350000
3400000
3450000
3500000
3550000
3600000
3650000
3700000
3750000
3800000
3850000
3900000
3950000
4000000
4050000
4100000
4150000
4200000
4250000
4300000
4350000
4400000
4450000
4500000
4550000
4600000
4650000
4700000
4750000
4800000
4850000
4900000
4950000
5000000
5050000
5100000
5150000
5200000
5250000
5300000
5350000
5400000
5450000
5500000
5550000
5600000
5650000
5700000
5750000
5800000
5850000
5900000
5950000
6000000
6050000
6100000
6150000
6200000
6250000
6300000
6350000
6400

ValueError: blocks[0,:] has incompatible row dimensions

In [12]:
joblib.dump(res, 'sparse_app_is_installed')
joblib.dump(events, 'events')

['events']